In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
from tqdm import tqdm

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
"""%%time
train_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')
test_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)"""

"%%time\ntrain_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')\ntest_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')\ntrain_metadata_kaggle = train_metadata_kaggle.compute()\ntest_metadata_kaggle = test_metadata_kaggle.compute()\nprint(train_metadata_kaggle.shape,test_metadata_kaggle.shape)"

In [5]:
%%time
train_metadata1 = dd.read_csv('train_metadata_final.csv')
test_metadata1 = dd.read_csv('test_metadata_final.csv')
train_metadata1 = train_metadata1.compute()
test_metadata1 = test_metadata1.compute()
print(train_metadata1.shape,test_metadata1.shape)

(7848, 233) (3492890, 232)
CPU times: user 3min 42s, sys: 13.3 s, total: 3min 56s
Wall time: 43.3 s


In [6]:
"""%%time
train_metadata2 = dd.read_csv('train_metadata_final.csv')
test_metadata2 = dd.read_csv('test_metadata_final.csv')
train_metadata2 = train_metadata2.compute()
test_metadata2 = test_metadata2.compute()
print(train_metadata2.shape,test_metadata2.shape)"""

"%%time\ntrain_metadata2 = dd.read_csv('train_metadata_final.csv')\ntest_metadata2 = dd.read_csv('test_metadata_final.csv')\ntrain_metadata2 = train_metadata2.compute()\ntest_metadata2 = test_metadata2.compute()\nprint(train_metadata2.shape,test_metadata2.shape)"

In [7]:
#columns1 = ['__amplitude___0_', '__amplitude___1_', '__amplitude___3_', '__amplitude___4_', '__amplitude___5_', '__flux_percentile_ratio_mid20___0_', '__flux_percentile_ratio_mid20___1_', '__flux_percentile_ratio_mid20___2_', '__flux_percentile_ratio_mid20___3_', '__flux_percentile_ratio_mid20___4_', '__flux_percentile_ratio_mid20___5_', '__flux_percentile_ratio_mid35___0_', '__flux_percentile_ratio_mid35___1_', '__flux_percentile_ratio_mid35___2_', '__flux_percentile_ratio_mid35___3_', '__flux_percentile_ratio_mid35___4_', '__flux_percentile_ratio_mid35___5_', '__flux_percentile_ratio_mid50___0_', '__flux_percentile_ratio_mid50___1_', '__flux_percentile_ratio_mid50___2_', '__flux_percentile_ratio_mid50___3_', '__flux_percentile_ratio_mid50___4_', '__flux_percentile_ratio_mid65___0_', '__flux_percentile_ratio_mid65___1_', '__flux_percentile_ratio_mid65___3_', '__flux_percentile_ratio_mid65___4_', '__flux_percentile_ratio_mid65___5_', '__flux_percentile_ratio_mid80___0_', '__flux_percentile_ratio_mid80___1_', '__flux_percentile_ratio_mid80___2_', '__flux_percentile_ratio_mid80___3_', '__flux_percentile_ratio_mid80___4_', '__flux_percentile_ratio_mid80___5_', '__max_slope___0_', '__max_slope___1_', '__max_slope___2_', '__max_slope___3_', '__max_slope___4_', '__max_slope___5_', '__linear_trend___0_', '__linear_trend___1_', '__linear_trend___2_', '__linear_trend___3_', '__linear_trend___4_', '__linear_trend___5_', '__median_absolute_deviation___0_', '__median_absolute_deviation___1_', '__median_absolute_deviation___3_', '__median_absolute_deviation___4_', '__median_absolute_deviation___5_', '__percent_close_to_median___0_', '__percent_close_to_median___1_', '__percent_close_to_median___3_', '__percent_close_to_median___4_', '__percent_close_to_median___5_', '__percent_difference_flux_percentile___0_', '__percent_difference_flux_percentile___1_', '__percent_difference_flux_percentile___2_', '__percent_difference_flux_percentile___3_', '__percent_difference_flux_percentile___4_', '__percent_difference_flux_percentile___5_', '__percent_amplitude___0_', '__percent_amplitude___1_', '__percent_amplitude___2_', '__percent_amplitude___3_', '__percent_amplitude___4_', '__percent_amplitude___5_', '__qso_log_chi2_qsonu___2_', '__qso_log_chi2_qsonu___4_', '__qso_log_chi2nuNULL_chi2nu___0_', '__qso_log_chi2nuNULL_chi2nu___1_', '__qso_log_chi2nuNULL_chi2nu___2_', '__qso_log_chi2nuNULL_chi2nu___3_', '__qso_log_chi2nuNULL_chi2nu___4_', '__qso_log_chi2nuNULL_chi2nu___5_', '__skew___0_', '__skew___1_', '__skew___2_', '__skew___3_', '__skew___4_', '__skew___5_', '__std___0_', '__std___1_', '__std___2_', '__std___3_', '__std___5_', '__stetson_j___0_', '__stetson_j___1_', '__stetson_j___2_', '__stetson_j___3_', '__stetson_j___4_', '__stetson_j___5_', '__stetson_k___0_', '__stetson_k___3_', '__stetson_k___4_', '__stetson_k___5_', '__freq1_amplitude1___0_', '__freq1_amplitude1___1_', '__freq1_amplitude1___2_', '__freq1_amplitude1___3_', '__freq1_amplitude1___4_', '__freq1_amplitude1___5_', '__freq1_amplitude2___0_', '__freq1_amplitude2___1_', '__freq1_amplitude2___2_', '__freq1_amplitude2___3_', '__freq1_amplitude2___4_', '__freq1_amplitude2___5_', '__freq1_amplitude3___0_', '__freq1_amplitude3___1_', '__freq1_amplitude3___2_', '__freq1_amplitude3___3_', '__freq1_amplitude3___4_', '__freq1_amplitude3___5_', '__freq1_amplitude4___0_', '__freq1_amplitude4___1_', '__freq1_amplitude4___2_', '__freq1_amplitude4___3_', '__freq1_amplitude4___4_', '__freq1_amplitude4___5_', '__freq1_freq___0_', '__freq1_freq___1_', '__freq1_freq___2_', '__freq1_freq___3_', '__freq1_freq___4_', '__freq1_freq___5_', '__freq1_rel_phase2___0_', '__freq1_rel_phase2___1_', '__freq1_rel_phase2___2_', '__freq1_rel_phase2___3_', '__freq1_rel_phase2___4_', '__freq1_rel_phase2___5_', '__freq1_rel_phase3___0_', '__freq1_rel_phase3___1_', '__freq1_rel_phase3___2_', '__freq1_rel_phase3___3_', '__freq1_rel_phase3___4_', '__freq1_rel_phase3___5_', '__freq1_rel_phase4___0_', '__freq1_rel_phase4___1_', '__freq1_rel_phase4___2_', '__freq1_rel_phase4___3_', '__freq1_rel_phase4___4_', '__freq1_rel_phase4___5_', '__freq2_amplitude1___0_', '__freq2_amplitude1___1_', '__freq2_amplitude1___2_', '__freq2_amplitude1___3_', '__freq2_amplitude1___4_', '__freq2_amplitude1___5_', '__freq2_amplitude2___0_', '__freq2_amplitude2___1_', '__freq2_amplitude2___2_', '__freq2_amplitude2___3_', '__freq2_amplitude2___4_', '__freq2_amplitude2___5_', '__freq2_amplitude3___0_', '__freq2_amplitude3___1_', '__freq2_amplitude3___2_', '__freq2_amplitude3___3_', '__freq2_amplitude3___4_', '__freq2_amplitude3___5_', '__freq2_amplitude4___0_', '__freq2_amplitude4___1_', '__freq2_amplitude4___2_', '__freq2_amplitude4___3_', '__freq2_amplitude4___4_', '__freq2_amplitude4___5_', '__freq2_freq___0_', '__freq2_freq___1_', '__freq2_freq___2_', '__freq2_freq___3_', '__freq2_freq___4_', '__freq2_freq___5_', '__freq2_rel_phase2___0_', '__freq2_rel_phase2___1_', '__freq2_rel_phase2___2_', '__freq2_rel_phase2___3_', '__freq2_rel_phase2___4_', '__freq2_rel_phase2___5_', '__freq2_rel_phase3___0_', '__freq2_rel_phase3___1_', '__freq2_rel_phase3___2_', '__freq2_rel_phase3___3_', '__freq2_rel_phase3___4_', '__freq2_rel_phase3___5_', '__freq2_rel_phase4___0_', '__freq2_rel_phase4___1_', '__freq2_rel_phase4___2_', '__freq2_rel_phase4___3_', '__freq2_rel_phase4___4_', '__freq2_rel_phase4___5_', '__freq3_amplitude1___0_', '__freq3_amplitude1___2_', '__freq3_amplitude1___3_', '__freq3_amplitude1___4_', '__freq3_amplitude1___5_', '__freq3_amplitude2___0_', '__freq3_amplitude2___1_', '__freq3_amplitude2___2_', '__freq3_amplitude2___3_', '__freq3_amplitude2___4_', '__freq3_amplitude2___5_', '__freq3_amplitude3___0_', '__freq3_amplitude3___1_', '__freq3_amplitude3___2_', '__freq3_amplitude3___3_', '__freq3_amplitude3___4_', '__freq3_amplitude3___5_', '__freq3_amplitude4___0_', '__freq3_amplitude4___1_', '__freq3_amplitude4___2_', '__freq3_amplitude4___3_', '__freq3_amplitude4___4_', '__freq3_amplitude4___5_', '__freq3_freq___0_', '__freq3_freq___1_', '__freq3_freq___2_', '__freq3_freq___3_', '__freq3_freq___4_', '__freq3_freq___5_', '__freq3_rel_phase2___0_', '__freq3_rel_phase2___1_', '__freq3_rel_phase2___2_', '__freq3_rel_phase2___3_', '__freq3_rel_phase2___4_', '__freq3_rel_phase2___5_', '__freq3_rel_phase3___0_', '__freq3_rel_phase3___1_', '__freq3_rel_phase3___2_', '__freq3_rel_phase3___3_', '__freq3_rel_phase3___4_', '__freq3_rel_phase3___5_', '__freq3_rel_phase4___0_', '__freq3_rel_phase4___1_', '__freq3_rel_phase4___2_', '__freq3_rel_phase4___3_', '__freq3_rel_phase4___4_', '__freq3_rel_phase4___5_', '__freq1_signif___0_', '__freq1_signif___1_', '__freq1_signif___3_', '__freq1_signif___4_', '__freq1_signif___5_', '__freq_signif_ratio_21___0_', '__freq_signif_ratio_21___1_', '__freq_signif_ratio_21___2_', '__freq_signif_ratio_21___3_', '__freq_signif_ratio_21___4_', '__freq_signif_ratio_21___5_', '__freq_signif_ratio_31___0_', '__freq_signif_ratio_31___1_', '__freq_signif_ratio_31___2_', '__freq_signif_ratio_31___3_', '__freq_signif_ratio_31___4_', '__freq_signif_ratio_31___5_', '__freq_varrat___0_', '__freq_varrat___1_', '__freq_varrat___2_', '__freq_y_offset___0_', '__freq_y_offset___1_', '__freq_y_offset___2_', '__freq_y_offset___3_', '__freq_y_offset___4_', '__freq_y_offset___5_']

In [8]:
columns2 = ['ra', 'decl', 'gal_l', 'gal_b', 'ddf', 'hostgal_specz', 'NG_min_flux_err', 'NG_max_flux_err', 'NG_std_flux_err', 'NG_sum_flux_err', 'NG_mean_flux_err', 'NG_median_flux_err', 'NG_min_flux', 'NG_max_flux', 'NG_std_flux', 'NG_sum_flux', 'NG_mean_flux', 'NG_median_flux', 'NG_count_detected', 'NG_std_detected', 'NG_sum_detected', 'NG_mean_detected', 'NG_median_detected', 'A0_min_flux_err', 'A0_max_flux_err', 'A0_std_flux_err', 'A0_sum_flux_err', 'A0_mean_flux_err', 'A0_median_flux_err', 'A0_min_flux', 'A0_max_flux', 'A0_std_flux', 'A0_median_flux', 'A0_count_detected', 'A0_sum_detected', 'A0_mean_detected', 'A0_median_detected', 'A1_min_flux_err', 'A1_max_flux_err', 'A1_std_flux_err', 'A1_sum_flux_err', 'A1_mean_flux_err', 'A1_median_flux_err', 'A1_min_flux', 'A1_max_flux', 'A1_std_flux', 'A1_sum_flux', 'A1_mean_flux', 'A1_median_flux', 'A1_count_detected', 'A1_std_detected', 'A1_sum_detected', 'A1_median_detected', 'A2_min_flux_err', 'A2_max_flux_err', 'A2_std_flux_err', 'A2_sum_flux_err', 'A2_mean_flux_err', 'A2_median_flux_err', 'A2_max_flux', 'A2_std_flux', 'A2_sum_flux', 'A2_mean_flux', 'A2_median_flux', 'A2_count_detected', 'A2_std_detected', 'A2_mean_detected', 'A2_median_detected', 'A3_min_flux_err', 'A3_max_flux_err', 'A3_std_flux_err', 'A3_sum_flux_err', 'A3_mean_flux_err', 'A3_median_flux_err', 'A3_min_flux', 'A3_max_flux', 'A3_std_flux', 'A3_sum_flux', 'A3_mean_flux', 'A3_median_flux', 'A3_count_detected', 'A3_std_detected', 'A3_sum_detected', 'A3_mean_detected', 'A3_median_detected', 'A4_min_flux_err', 'A4_max_flux_err', 'A4_std_flux_err', 'A4_sum_flux_err', 'A4_mean_flux_err', 'A4_median_flux_err', 'A4_min_flux', 'A4_max_flux', 'A4_std_flux', 'A4_sum_flux', 'A4_mean_flux', 'A4_median_flux', 'A4_count_detected', 'A4_std_detected', 'A4_sum_detected', 'A4_median_detected', 'A5_min_flux_err', 'A5_max_flux_err', 'A5_std_flux_err', 'A5_sum_flux_err', 'A5_mean_flux_err', 'A5_median_flux_err', 'A5_min_flux', 'A5_max_flux', 'A5_std_flux', 'A5_sum_flux', 'A5_mean_flux', 'A5_median_flux', 'A5_count_detected', 'A5_median_detected', 'p0_region_minus_4', 'p0_region_minus_3', 'p0_region_minus_2', 'p0_region_minus_1', 'p0_region_plus_1', 'p0_region_plus_2', 'p0_region_plus_3', 'p0_region_plus_4', 'p1_region_minus_4', 'p1_region_minus_3', 'p1_region_minus_2', 'p1_region_minus_1', 'p1_region_plus_1', 'p1_region_plus_2', 'p1_region_plus_3', 'p1_region_plus_4', 'p2_region_minus_4', 'p2_region_minus_3', 'p2_region_minus_2', 'p2_region_minus_1', 'p2_region_plus_1', 'p2_region_plus_2', 'p2_region_plus_3', 'p2_region_plus_4', 'p3_region_minus_4', 'p3_region_minus_3', 'p3_region_minus_2', 'p3_region_minus_1', 'p3_region_plus_1', 'p3_region_plus_2', 'p3_region_plus_3', 'p3_region_plus_4', 'p4_region_minus_4', 'p4_region_minus_3', 'p4_region_minus_2', 'p4_region_minus_1', 'p4_region_plus_1', 'p4_region_plus_2', 'p4_region_plus_3', 'p4_region_plus_4', 'p5_region_minus_4', 'p5_region_minus_3', 'p5_region_minus_2', 'p5_region_minus_1', 'p5_region_plus_1', 'p5_region_plus_2', 'p5_region_plus_3', 'p5_region_plus_4', 'number_of_0s', 'number_of_1s', 'number_of_2s', 'number_of_3s', 'number_of_4s', 'number_of_5s', 'percent_p0_region_minus_4', 'percent_p0_region_minus_3', 'percent_p0_region_minus_2', 'percent_p0_region_minus_1', 'percent_p0_region_plus_1', 'percent_p0_region_plus_2', 'percent_p0_region_plus_3', 'percent_p0_region_plus_4', 'percent_p1_region_minus_4', 'percent_p1_region_minus_3', 'percent_p1_region_minus_2', 'percent_p1_region_minus_1', 'percent_p1_region_plus_1', 'percent_p1_region_plus_2', 'percent_p1_region_plus_3', 'percent_p1_region_plus_4', 'percent_p2_region_minus_4', 'percent_p2_region_minus_3', 'percent_p2_region_minus_2', 'percent_p2_region_plus_1', 'percent_p2_region_plus_2', 'percent_p2_region_plus_3', 'percent_p2_region_plus_4', 'percent_p3_region_minus_4', 'percent_p3_region_minus_3', 'percent_p3_region_minus_2', 'percent_p3_region_minus_1', 'percent_p3_region_plus_1', 'percent_p3_region_plus_2', 'percent_p3_region_plus_3', 'percent_p3_region_plus_4', 'percent_p4_region_minus_4', 'percent_p4_region_minus_3', 'percent_p4_region_minus_2', 'percent_p4_region_minus_1', 'percent_p4_region_plus_1', 'percent_p4_region_plus_2', 'percent_p4_region_plus_3', 'percent_p4_region_plus_4', 'percent_p5_region_minus_4', 'percent_p5_region_minus_3', 'percent_p5_region_minus_2', 'percent_p5_region_minus_1', 'percent_p5_region_plus_1', 'percent_p5_region_plus_2', 'percent_p5_region_plus_3', 'percent_p5_region_plus_4']

In [ ]:
#print(train_metadata2.shape,test_metadata2.shape)

In [ ]:
#columns1 = [x for x in train_metadata1 if x not in train_metadata_kaggle.columns]
#columns2 = [x for x in train_metadata2 if x not in train_metadata_kaggle.columns]

In [ ]:
#print(columns2)

In [ ]:
#print(len(columns1),len(columns2))

In [9]:
columns2[:5]

['ra', 'decl', 'gal_l', 'gal_b', 'ddf']

In [10]:
%%time
train_metadata1 = train_metadata1[columns2 + ['object_id']]
#train_metadata2 = train_metadata2[columns2 + ['object_id']]
test_metadata1 = test_metadata1[columns2 + ['object_id']]
#test_metadata2 = test_metadata2[columns2 + ['object_id']]

CPU times: user 3.75 s, sys: 5.83 s, total: 9.58 s
Wall time: 9.57 s


In [11]:
print(train_metadata1.shape,test_metadata1.shape)

(7848, 217) (3492890, 217)


In [12]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [13]:
%%time
train_metadata1 = reduce_mem_usage(train_metadata1)
test_metadata1 = reduce_mem_usage(test_metadata1)
#train_metadata2 = reduce_mem_usage(train_metadata2)
#test_metadata2 = reduce_mem_usage(test_metadata2)

Memory usage of dataframe is 12.99 MB
Memory usage after optimization is: 3.88 MB
Decreased by 70.1%
Memory usage of dataframe is 5809.40 MB
Memory usage after optimization is: 1815.44 MB
Decreased by 68.8%
CPU times: user 2min 20s, sys: 3min 11s, total: 5min 32s
Wall time: 5min 32s


In [14]:
print(train_metadata1.shape,test_metadata1.shape)

(7848, 217) (3492890, 217)


In [15]:
test_metadata1.tail()

,ra,decl,gal_l,gal_b,ddf,hostgal_specz,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_median_flux,A0_count_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_plus_1,percent_p5_region_plus_2,percent_p5_region_plus_3,percent_p5_region_plus_4,object_id
15069,67.500000,-23.812500,221.875,-40.937500,0,NaN,1.340820,70.066055,12.628469,2152.682373,14.846085,10.953125,-49.490440,75.235832,21.009541,566.28735

In [16]:
%%time
test_metadata1 = pd.concat([test_metadata1,train_metadata1],axis=0)

CPU times: user 2.93 s, sys: 1.67 s, total: 4.6 s
Wall time: 4.6 s


In [17]:
%%time
test_metadata1 = test_metadata1.reset_index(drop=True)

CPU times: user 1.14 s, sys: 1.51 s, total: 2.65 s
Wall time: 2.65 s


In [18]:
%%time
test_metadata1.replace(np.inf, np.nan,inplace=True)
test_metadata1.replace(-np.inf, np.nan,inplace=True)

CPU times: user 13.5 s, sys: 74.6 ms, total: 13.5 s
Wall time: 13.5 s


In [19]:
%%time
test_metadata1.fillna(test_metadata1.mean(),inplace=True)

CPU times: user 11.9 s, sys: 3.71 s, total: 15.6 s
Wall time: 15.6 s


In [20]:
print(test_metadata1.shape)

(3500738, 217)


In [21]:
test_metadata1.tail()

,ra,decl,gal_l,gal_b,ddf,hostgal_specz,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_median_flux,A0_count_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_plus_1,percent_p5_region_plus_2,percent_p5_region_plus_3,percent_p5_region_plus_4,object_id
3500733,26.71875,-14.937500,172.375000,-72.250000,0,0.000000,1.701172,72.230759,14.645406,2409.608887,16.391897,11.132812,-105.375282,517.602478,50.60094

In [22]:
%%time
test_metadata1.fillna(0.0,inplace=True)

CPU times: user 1.87 s, sys: 0 ns, total: 1.87 s
Wall time: 1.86 s


In [23]:
object_ids = test_metadata1['object_id']
del test_metadata1['object_id']

In [24]:
print(test_metadata1.shape)

(3500738, 216)


In [25]:
from sklearn.preprocessing import MinMaxScaler

In [26]:
scaler = MinMaxScaler()

In [27]:
%%time
for i in test_metadata1:
    #print(i)
    test_metadata1[i].replace(np.inf, 0, inplace=True)
    test_metadata1[i].replace(-np.inf, 0, inplace=True)

CPU times: user 11.7 s, sys: 13 ms, total: 11.7 s
Wall time: 11.7 s


In [28]:
%%time
scaler.fit(test_metadata1)

CPU times: user 6.43 s, sys: 3.87 s, total: 10.3 s
Wall time: 10.3 s


MinMaxScaler(copy=True, feature_range=(0, 1))

In [29]:
%%time
test_metadata1 = scaler.transform(test_metadata1)

CPU times: user 6.62 s, sys: 3.3 s, total: 9.92 s
Wall time: 9.92 s


In [30]:
# Training Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 256
# Network Parameters
num_hidden_1 = 128 # 1st layer num features
num_hidden_2 = 32 # 2nd layer num features (the latent dim)
num_input = test_metadata1.shape[1] # MNIST data input (img shape: 28*28)

In [31]:
tf.reset_default_graph()
# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
}

# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
loss_op = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_op)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [32]:
X_train = test_metadata1[:int(len(test_metadata1)*0.8)]
X_cv = test_metadata1[int(len(test_metadata1)*0.8):]

In [33]:
remaining = X_train.shape[0] % batch_size

In [34]:
# Start a new TF session
saver = tf.train.Saver()
#GPU settings
config = tf.ConfigProto(allow_soft_placement=True)

config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'

with tf.Session(config=config) as sess:
    #Runnig initialization of variables
    sess.run(init)
    # Writing down the for loop for epochs
    for epoch in range(training_epochs):
        #For loop for batches
        for i in range(0,X_train.shape[0] - remaining,batch_size):
            #Getting training data to be fed into the graphs
            batch_x= X_train[i:i+batch_size]
            # Training batch by batch
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x})
        
        k = 0
        arr = np.empty((0,X_train.shape[1]), float)
        loss_list = []
        for j in range(0,len(X_train),len(X_cv)):
            loss_on_train = sess.run(loss_op,feed_dict = {X: X_train[ j: j + len(X_cv) ] } )
            loss_list.append(loss_on_train)
        
        #np.append(arr, np.array([[1,2,3]]), axis=0)
        #loss_on_train= sess.run(loss_op,feed_dict = {X: X_train})
        loss_on_cv= sess.run(loss_op,feed_dict = {X: X_cv})
        print(epoch, "Train error = ",round( np.mean(loss_list),6 ) )
        print(epoch, "CV error = ", loss_on_cv )

    saver.save(sess, os.path.join(os.getcwd(),"AE_cesium.ckpt"))
    
    embedding_all = np.empty((0,num_hidden_2), float)
    
    for j in range(0,len(test_metadata1),len(X_cv)):
        embedding_all_ = sess.run(encoder_op, feed_dict={X: test_metadata1[j: j + len(X_cv)] })
        embedding_all = np.append(embedding_all, embedding_all_, axis=0)
    print(embedding_all.shape)

0 Train error =  0.016919
0 CV error =  0.016721472
1 Train error =  0.013177
1 CV error =  0.012983814
2 Train error =  0.012774
2 CV error =  0.012568268
3 Train error =  0.012619
3 CV error =  0.012412525
4 Train error =  0.011684
4 CV error =  0.011481102
5 Train error =  0.009527
5 CV error =  0.009344414
6 Train error =  0.009498
6 CV error =  0.009322371
7 Train error =  0.009517
7 CV error =  0.009346465
8 Train error =  0.009619
8 CV error =  0.009454298
9 Train error =  0.007353
9 CV error =  0.007180535
(3500738, 32)


In [35]:
%%time
embedding_train = pd.DataFrame(embedding_all[3492890:],columns = ['e_' + str(x + 1) for x in range(num_hidden_2) ])
embedding_test = pd.DataFrame(embedding_all[:3492890],columns=['e_' + str(x + 1) for x in range(num_hidden_2) ])

CPU times: user 1.31 ms, sys: 565 µs, total: 1.87 ms
Wall time: 1.54 ms


In [36]:
print(embedding_train.shape,embedding_test.shape)

(7848, 32) (3492890, 32)


In [37]:
train_id = object_ids[3492890:]
train_id = train_id.reset_index(drop=True)

test_id = object_ids[:3492890]
test_id = test_id.reset_index(drop=True)

In [38]:
%%time
embedding_train = pd.concat([train_id,embedding_train],axis=1)
embedding_test = pd.concat([test_id,embedding_test],axis=1)

CPU times: user 1.11 s, sys: 227 ms, total: 1.34 s
Wall time: 1.34 s


In [39]:
print(embedding_train.shape,embedding_test.shape)

(7848, 33) (3492890, 33)


In [42]:
embedding_train.columns = ['object_id'] + ['f_' + str(x + 1) for x in range(num_hidden_2) ]
embedding_test.columns = ['object_id'] + ['f_' + str(x + 1) for x in range(num_hidden_2) ]

In [43]:
embedding_train.head()

,object_id,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,f_12,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,f_31,f_32
0,615,1.0,0.164372,0.222792,5.438605e-09,0.651812,0.999989,0.606113,0.999999,0.135571,0.000336,0.621347,0.827612,0.999994,0.999976,0.561537,0.257014,0.052753,0.999985,0.537093,0.392824,0.486611,0.875413,0.693026,1.835425e-07,1.0,0.999999,0.847907,0.998799,0.999306,0.000051,0.833349,0.291361
1,713,1.0,0.252074,0.108779,1.917341e-09,0.619971,0.999984,0.376143,0.999983,0.102502,0.000100,0.638736,0.819630,0.999996,0.999996,0.733509,0.071764,0.715515,0.999992,0.194113,0.281997,0.439453,0.763622,0.769998,4.358771e-09,1.0,0.999999,0.882605,0.998602,0.995480,0.000065,0.781426,0.185945
2,730,1.0,0.568631,0.520688,4.884106e-09,0.741966,0.999997,0.471407,0.999982,0.608294,0.000060,0.596208,0.919730,0.999999,0.999987,0.756239,0.080395,0.710038,0.999997,0.243448,0.327783,0.335794,0.360859,0.828025,1.433233e-07,1.0,1.000000,0.724177,0.995608,0.998718,0.000025,0.758889,0.246271
3,745,1.0,0.601479,0.454393,1.214281e-09,0.648169,0.999999,0.289976,0.999917,0.038940,0.000036,0.581961,0.841444,1.000000,0.999983,0.840447,0.047467,0.677342,0.999999,0.143827,0.259299,0.287243,0.289878,0.887273,5.742507e-08,1.0,0.999998,0.719049,0.996194,0.997235,0.000022,0.734991,0.451858
4,1124,1.0,0.573290,0.410976,4.495547e-09,0.447952,0.999992,0.336990,0.999965,0.045460,0.000261,0.580087,0.842975,1.000000,0.999993,0.816532,0.244958,0.667996,0.999999,0.360816,0.171212,0.376273,0.331482,0.765855,2.061060e-08,1.0,0.999999,0.760879,0.999007,0.990677,0.000009,0.647324,0.500192


In [44]:
%%time
embedding_train.to_csv('myfeatures_train_embeddings.csv',index=False)
embedding_test.to_csv('myfeatures_test_embeddings.csv',index=False)

CPU times: user 3min 3s, sys: 1.59 s, total: 3min 5s
Wall time: 3min 4s
